## 1. 데이터 분석 연습하기
고객 3,500명에 대한 학습용 데이터를 이용하여 성별 예측 모형을 만든 후, 이를 평가용 데이터에 적용하여 얻은 2,482명 고객의 성별 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.
(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

### 1.1 데이터 탐색하기

In [1]:
# pandas 라이브러리 가져오기
import pandas as pd
# 주어진 데이터 파일을 모두 읽어서, 데이터 프레임 변수에 저장하기

x_train = pd.read_csv('bigData-main/x_train.csv', encoding='ms949')
x_test = pd.read_csv('bigData-main/x_test.csv', encoding='ms949')
y_train = pd.read_csv('bigData-main/y_train.csv', encoding='ms949')

In [2]:
print(x_train.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  


In [3]:
print(x_test.head())

   cust_id       총구매액     최대구매액        환불금액 주구매상품 주구매지점  내점일수    내점당구매건수  \
0     3500   70900400  22000000   4050000.0    골프  부산본점    13   1.461538   
1     3501  310533100  38558000  48034700.0   농산물   잠실점    90   2.433333   
2     3502  305264140  14825000  30521000.0  가공식품  본  점   101  14.623762   
3     3503    7594080   5225000         NaN  주방용품  부산본점     5   2.000000   
4     3504    1795790   1411200         NaN   수산품  청량리점     3   2.666667   

     주말방문비율  구매주기  
0  0.789474    26  
1  0.369863     3  
2  0.083277     3  
3  0.000000    47  
4  0.125000     8  


In [4]:
print(y_train.head())

   cust_id  gender
0        0       0
1        1       0
2        2       1
3        3       1
4        4       0


#### 말 줄임표로 확인이 어려운 데이터를 확인하는 방법
(1) T, transpose() 함수 사용해서 열, 행 방향 바꾸기  
(2) 전체 칼럼을 출력하는 옵션 사용하기
 - pd.options.display.max_columns = None

In [5]:
pd.options.display.max_columns = None
print(x_train.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  


In [6]:
# 행 / 열 확인하기
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(3500, 10)
(2482, 10)
(3500, 2)


In [7]:
# 요약 정보 확인하기
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB
None


In [8]:
# 기초 통계량 확인하기
print(x_train.describe().T)

          count          mean           std         min           25%  \
cust_id  3500.0  1.749500e+03  1.010507e+03         0.0  8.747500e+02   
총구매액     3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액    3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
환불금액     1205.0  2.407822e+07  4.746453e+07      5600.0  2.259000e+06   
내점일수     3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수  3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율   3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기     3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   

                  50%           75%           max  
cust_id  1.749500e+03  2.624250e+03  3.499000e+03  
총구매액     2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액    9.837000e+06  2.296250e+07  7.066290e+08  
환불금액     7.392000e+06  2.412000e+07  5.637530e+08  
내점일수     8.000000e+00  2.500000e+01  2.850000e+02  
내점당구매건수  2.333333e+00  3.37500

### 1.2 전처리하기

In [9]:
# 테스트 데이터의 cust_id 값은 x_test_cust_id 변수에 저장하기
# 분석과정에는 불필요하나 추후 제출에 필요하다
x_test_cust_id = x_test['cust_id']

In [10]:
# cust_id 칼럼을 삭제하기
x_train = x_train.drop(columns=['cust_id'])
y_train = y_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])

# 칼럼이 삭제된 상위 5개 행을 확인하기
print(x_train.head())
print(y_train.head())

       총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수    주말방문비율  구매주기
0  68282840  11264000  6860000.0      기타   강남점    19  3.894737  0.527027    17
1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000  0.000000     1
2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000  0.000000     1
3  16077620   4935000        NaN      기타   광주점    18  2.444444  0.318182    16
4  29050000  24000000        NaN      보석  본  점     2  1.500000  0.000000    85
   gender
0       0
1       0
2       1
3       1
4       0


In [11]:
# 결측치 처리하기
print(x_train.isnull().sum())

총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [12]:
# '환불금액' 칼럼의 결측치는 0으로 대치하기
x_train['환불금액'] = x_train['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)
# 결측치가 조치되었는지, '환불금액' 칼럼의 결측치 개수를 확인하기
print(x_train['환불금액'].isnull().sum())
print(x_test['환불금액'].isnull().sum())

0
0


In [13]:
# 범주형 변수를 인코딩하기

# '주구매상품' 칼럼에서 중복을 제외한 값들을 확인하기
print(x_train['주구매상품'].unique())
# '주구매상품' 칼럼에서 중복을 제외한 값들의 개수 세기
print(x_train['주구매상품'].unique().size)
# '주구매지점' 칼럼에서 중복을 제외한 값들을 확인하기
print(x_train['주구매지점'].unique())
# '주구매지점' 칼럼에서 중복을 제외한 값들의 개수 세기
print(x_train['주구매지점'].unique().size)


['기타' '스포츠' '남성 캐주얼' '보석' '디자이너' '시티웨어' '명품' '농산물' '화장품' '골프' '구두' '가공식품'
 '수산품' '아동' '차/커피' '캐주얼' '섬유잡화' '육류' '축산가공' '젓갈/반찬' '액세서리' '피혁잡화' '일용잡화'
 '주방가전' '주방용품' '건강식품' '가구' '주류' '모피/피혁' '남성 트랜디' '셔츠' '남성정장' '생활잡화'
 '트래디셔널' '란제리/내의' '커리어' '침구/수예' '대형가전' '통신/컴퓨터' '식기' '소형가전' '악기']
42
['강남점' '잠실점' '관악점' '광주점' '본  점' '일산점' '대전점' '부산본점' '분당점' '영등포점' '미아점'
 '청량리점' '안양점' '부평점' '동래점' '포항점' '노원점' '창원점' '센텀시티점' '인천점' '대구점' '전주점'
 '울산점' '상인점']
24


In [14]:
from sklearn.preprocessing import LabelEncoder
# 라벨 인코딩 수행을 위한 encoder 함수 만들기
encoder = LabelEncoder()

In [15]:
# '주구매상품'에 대해 라벨 인코딩을 수행한 후, '주구매상품'칼럼으로 다시 저장하기
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
# 라벨인코딩 결과를 확인하기 위해, 상위 10개 행을 확인하기
print(x_train['주구매상품'].head(10))

0     5
1    21
2     6
3     5
4    15
5    11
6    22
7    13
8     5
9     9
Name: 주구매상품, dtype: int32


In [16]:
# '주구매상품' 칼럼에 대한 라벨 인코딩의 변환 순서 확인하기
print(encoder.classes_)

# 테스트 데이터로 라벨 인코딩 수행하기
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
print(encoder.classes_)

['가공식품' '가구' '건강식품' '골프' '구두' '기타' '남성 캐주얼' '남성 트랜디' '남성정장' '농산물' '대형가전'
 '디자이너' '란제리/내의' '명품' '모피/피혁' '보석' '생활잡화' '섬유잡화' '셔츠' '소형가전' '수산품' '스포츠'
 '시티웨어' '식기' '아동' '악기' '액세서리' '육류' '일용잡화' '젓갈/반찬' '주류' '주방가전' '주방용품'
 '차/커피' '축산가공' '침구/수예' '캐주얼' '커리어' '통신/컴퓨터' '트래디셔널' '피혁잡화' '화장품']
['가공식품' '가구' '건강식품' '골프' '구두' '기타' '남성 캐주얼' '남성 트랜디' '남성정장' '농산물' '대형가전'
 '디자이너' '란제리/내의' '명품' '모피/피혁' '보석' '생활잡화' '섬유잡화' '셔츠' '수산품' '스포츠' '시티웨어'
 '식기' '아동' '악기' '액세서리' '육류' '일용잡화' '젓갈/반찬' '주류' '주방가전' '주방용품' '차/커피'
 '축산가공' '침구/수예' '캐주얼' '커리어' '통신/컴퓨터' '트래디셔널' '피혁잡화' '화장품']


In [24]:
######### 물어볼 내용 ###########
# fit_transform 은 처음에만하고
# test 데이터에는 transform 만 해야하나?
print(x_train['주구매상품'].value_counts().sort_index())
print(x_test['주구매상품'].value_counts().sort_index())

0     546
1      10
2      47
3      82
4      54
5     595
6      55
7       2
8      22
9     339
10      8
11    193
12      8
13    100
14     57
15      3
16     15
17     98
18     30
19      2
20    153
21     69
22    213
23      7
24     40
25      2
26      5
27     57
28     64
29     29
30     14
31     26
32     32
33     44
34     35
35      4
36    101
37      9
38      3
39     23
40     40
41    264
Name: 주구매상품, dtype: int64
0     395
1       7
2      36
3      47
4      44
5     465
6      46
7       4
8      25
9     235
10      6
11    123
12     10
13     60
14     47
15      1
16      8
17     72
18     25
19     86
20     43
21    168
22      8
23     24
24      1
25      7
26     38
27     30
28     18
29     10
30     10
31     21
32     55
33     28
34      3
35     65
36      6
37      3
38      9
39     16
40    177
Name: 주구매상품, dtype: int64


In [17]:
# '주구매지점'에 대해 라벨 인코딩을 수행한 후, '주구매지점' 칼럼으로 다시 저장하기
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])

# 라벨인코딩 결과를 확인하기 위해, 상위 10개 행을 확인하기
print(x_train['주구매지점'].head(10))

0     0
1    19
2     1
3     2
4     8
5    18
6     0
7     8
8     8
9     5
Name: 주구매지점, dtype: int32


In [18]:
# '주구매지점' 칼럼에 대한 라벨 인코딩의 변환 순서 확인하기
print(encoder.classes_)
# 테스트 데이터의 '주구매지점' 칼럼도 라벨 인코딩 수행하기
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])
print(encoder.classes_)


['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']
['강남점' '관악점' '광주점' '노원점' '대구점' '대전점' '동래점' '미아점' '본  점' '부산본점' '부평점' '분당점'
 '상인점' '센텀시티점' '안양점' '영등포점' '울산점' '인천점' '일산점' '잠실점' '전주점' '창원점' '청량리점'
 '포항점']


In [26]:
# '환불금액' 칼럼이 0보다 큰지에 대한 조건을 condition 변수에 저장하기
condition = x_train['환불금액'] > 0

In [32]:
# condition 조건에 맞으면 '환불금액_new' 칼럼을 1로 설정하기
x_train.loc[condition,'환불금액_new'] = 1
# condition 조건이 맞지 않으면, '환불금액_new' 칼럼을 0으로 설정하기
x_train.loc[~condition,'환불금액_new'] = 0

In [34]:
# '환불금액', '환불금액_new' 칼럼을 비교하기
print(x_train[['환불금액', '환불금액_new']])

           환불금액  환불금액_new
0     6860000.0       1.0
1      300000.0       1.0
2           0.0       0.0
3           0.0       0.0
4           0.0       0.0
...         ...       ...
3495        0.0       0.0
3496  6049600.0       1.0
3497        0.0       0.0
3498        0.0       0.0
3499  5973000.0       1.0

[3500 rows x 2 columns]


In [35]:
# '환불금액'칼럼의 삭제 내용은 바로 데이터 세트에 반영하기
x_train = x_train.drop(columns=['환불금액'])

In [36]:
# 테스트 데이터 세트에서도 '환불금액_new'라는 파생변수 만들기(~는 not의 의미)
x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0

# '환불금액' 칼럼의 삭제 내용은 바로 데이터 세트에 반영하기
x_test = x_test.drop(columns=['환불금액'])

In [37]:
# 크기변환 전, x_train 세트의 기초 통계량 확인하기
print(x_train.describe().T)

           count          mean           std         min           25%  \
총구매액      3500.0  9.191925e+07  1.635065e+08 -52421520.0  4.747050e+06   
최대구매액     3500.0  1.966424e+07  3.199235e+07  -2992000.0  2.875000e+06   
주구매상품     3500.0  1.461200e+01  1.301995e+01         0.0  5.000000e+00   
주구매지점     3500.0  1.073429e+01  5.636480e+00         0.0  8.000000e+00   
내점일수      3500.0  1.925371e+01  2.717494e+01         1.0  2.000000e+00   
내점당구매건수   3500.0  2.834963e+00  1.912368e+00         1.0  1.666667e+00   
주말방문비율    3500.0  3.072463e-01  2.897516e-01         0.0  2.729090e-02   
구매주기      3500.0  2.095829e+01  2.474868e+01         0.0  4.000000e+00   
환불금액_new  3500.0  3.442857e-01  4.752027e-01         0.0  0.000000e+00   

                   50%           75%           max  
총구매액      2.822270e+07  1.065079e+08  2.323180e+09  
최대구매액     9.837000e+06  2.296250e+07  7.066290e+08  
주구매상품     9.000000e+00  2.200000e+01  4.100000e+01  
주구매지점     9.000000e+00  1.500000e+01  2.300000e

In [38]:
# sklearn 패키지의 preprocessing 모듈에서 StandardScaler 함수를 가져오기
from sklearn.preprocessing import StandardScaler
# 표준화 크기 변환을 수행하기 위한 scaler 객체를 만들기
scaler = StandardScaler()
# scaler 객체로 표준화 크기변환을 수행하고, x_train의 칼럼명을 사용하기
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)

In [39]:
# 테스트 데이터는 transform() 함수만으로 값을 변환
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

In [41]:
# 크기 변환 후, x_train 세트의 기초 통계량 확인하기
print(x_train.describe().T)

           count          mean       std       min       25%       50%  \
총구매액      3500.0 -4.409171e-17  1.000143 -0.882909 -0.533218 -0.389621   
최대구매액     3500.0 -4.838986e-17  1.000143 -0.708278 -0.524864 -0.307219   
주구매상품     3500.0 -6.236281e-17  1.000143 -1.122438 -0.738357 -0.431093   
주구매지점     3500.0 -8.333017e-17  1.000143 -1.904703 -0.485175 -0.307733   
내점일수      3500.0  2.518303e-16  1.000143 -0.671807 -0.635003 -0.414180   
내점당구매건수   3500.0 -2.288170e-16  1.000143 -0.959661 -0.611003 -0.262346   
주말방문비율    3500.0  9.192647e-17  1.000143 -1.060530 -0.966329 -0.175472   
구매주기      3500.0 -1.220294e-16  1.000143 -0.846966 -0.685318 -0.321610   
환불금액_new  3500.0  3.445498e-16  1.000143 -0.724606 -0.724606 -0.724606   

               75%        max  
총구매액      0.089237  13.648260  
최대구매액     0.103110  21.475852  
주구매상품     0.567518   2.027026  
주구매지점     0.756913   2.176441  
내점일수      0.211486   9.780490  
내점당구매건수   0.282432  10.066639  
주말방문비율    0.489224   2.391196  
구매주

In [42]:
# '총구매액', '최대구매액', '환불금액_new' 칼럼 간의 상관관계 구하기
print(x_train[['총구매액', '최대구매액', '환불금액_new']].corr())

              총구매액     최대구매액  환불금액_new
총구매액      1.000000  0.700080  0.403357
최대구매액     0.700080  1.000000  0.330687
환불금액_new  0.403357  0.330687  1.000000


In [43]:
# x_train 세트에서 '최대구매액' 칼럼을 삭제한 후, x_train에 저장하기
x_train = x_train.drop(columns=['최대구매액'])
# x_test 세트에서 '최대구매액' 칼럼을 삭제한 후, x_test에 저장하기
x_test = x_test.drop(columns=['최대구매액'])

### 1.3 학습하고 평가하기